In [1]:
import time
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

from tqdm import tqdm
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

/home/yizhenjia/anaconda3/envs/lmflow/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("/home/yizhenjia/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-70B/snapshots/b4d08b7db49d488da3ac49adf25a6b9ac01ae338")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = AutoModelForCausalLM.from_pretrained(
    '/home/yizhenjia/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-70B/snapshots/b4d08b7db49d488da3ac49adf25a6b9ac01ae338',
    device_map='auto',
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
)

Loading checkpoint shards: 100%|██████████| 30/30 [01:52<00:00,  3.77s/it]


In [4]:
user_input_ids = tokenizer.encode(
    'Write a function',
    return_tensors='pt'
).to(model.device)

In [7]:
total_generated_tokens = 0
start_time = time.time()
while True:
    res = model.generate(
        user_input_ids,
        max_length=256,
        temperature=1.0
    )
    total_generated_tokens += (res.shape[1] - user_input_ids.shape[1])
    if total_generated_tokens >= 1000:
        break
total_time = time.time() - start_time

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

In [8]:
model = PeftModel.from_pretrained(
    model,
    '/vol/yizhenjia/projs/multilora-speedtest/lora/adapter_model_1'
)

In [9]:
total_generated_tokens_lora = 0
start_time_lora = time.time()
while True:
    res = model.generate(
        user_input_ids,
        max_length=256,
        temperature=1.0
    )
    total_generated_tokens_lora += (res.shape[1] - user_input_ids.shape[1])
    if total_generated_tokens_lora >= 1000:
        break
total_time_lora = time.time() - start_time_lora

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

In [10]:
print(f'original: {total_generated_tokens} tokens in {total_time:.2f}s')
print(f'original speed: {total_generated_tokens / total_time:.2f} tokens/s')
print(f'lora: {total_generated_tokens_lora} tokens in {total_time_lora:.2f}s')
print(f'lora speed: {total_generated_tokens_lora / total_time_lora:.2f} tokens/s')

original: 1035 tokens in 271.92s
original speed: 3.81 tokens/s
lora: 1000 tokens in 264.11s
lora speed: 3.79 tokens/s
